In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import os
import ctypes
libc = ctypes.CDLL("libc.so.6")
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [ ]:
import warnings

# FutureWarning 및 UserWarning을 한 번만 출력
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=UserWarning)

In [ ]:
# !conda install -c pytorch -c nvidia faiss-gpu=1.9.0 --yes
!pip install -qU --no-deps /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install -q flash-attn
# !pip install -q vllm
!pip install -qU bitsandbytes
!pip install -qU peft
!pip install -qU trl

In [ ]:
!nvidia-smi

Wed Oct 23 19:19:05 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             12W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
torch.cuda.device_count()

2

In [ ]:
import transformers
from transformers import (
    AutoModel,
    AutoModelForCausalLM,
    AutoModelForMultipleChoice,
    AutoTokenizer,
    AutoConfig,
    GenerationConfig,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
)

In [ ]:
MODEL_PATH = '/kaggle/input/llama-3.2/transformers/1b-instruct/1'

In [ ]:
# quantization_config = BitsAndBytesConfig(load_in_8bit=True) 8-bit quantization
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit = True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_use_double_quant=True,
# ) #4-bit quantization
# model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, quantization_config=quantization_config, device_map="cuda:0")

model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)
model = model.to('cuda:0')
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
print(model.device)

cuda:0


In [ ]:
# messages = [{"role": "user", "content": "The key to life is"}]
# prompt = tokenizer.apply_chat_template(messages, tokenize=False)
prompt = "The key to life is"
print(prompt)
inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
output = model.generate(**inputs, max_new_tokens=32)[0]
tokenizer.decode(output)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The key to life is


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


'The key to life is not to be found in the end, but in the journey. - Robert Frost\n\nI hope this resonates with you and inspires you to take the next step'

In [ ]:
df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
df

,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D
...,...,...,...,...,...,...,...,...
195,195,What is the relation between the three moment ...,The three moment theorem expresses the relatio...,The three moment theorem is used to calculate ...,The three moment theorem describes the relatio...,The three moment theorem is used to calculate ...,The three moment theorem is used to derive the...,C
196,196,"What is the throttling process, and why is it ...",The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,The throttling process is a steady adiabatic f...,The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,B
197,197,What happens to excess base metal as a solutio...,"The excess base metal will often solidify, bec...",The excess base metal will often crystallize-o...,"The excess base metal will often dissolve, bec...","The excess base metal will often liquefy, beco...","The excess base metal will often evaporate, be...",B
198,198,"What is the relationship between mass, force, ...",Mass is a property that determines the weight ...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is a property that determines the size of...,D


# Zero-shot Prompting

In [ ]:
#Next Token Predicton + greedy decoding
preds = []
answers = []
for i in tqdm(range(len(df))):
    row = df.iloc[i]
    # print(row.values)
    question = row['prompt']
    answer = row['answer']
    a, b, c, d, e = row['A'], row['B'], row['C'], row['D'], row['E']
    prompt = f'{question}\nA: {a}\nB: {b}\nC: {c}\nD: {d}\nE: {e}\n\nThe answer is:'
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    pred = model.generate(**inputs, max_new_tokens=1, do_sample=False)[0]
    pred = tokenizer.decode(pred)
    if i < 10:
        print(pred)
        print(f'Ground Truth: {answer}')
    preds.append(pred[-1])
    answers.append(answer)
print('correct:', (np.array(answers) == np.array(preds)).sum(), '/', len(df))

  1%|          | 2/200 [00:00<00:36,  5.43it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Which of the following statements accurately describes the impact of Modified Newtonian Dynamics (MOND) on the observed "missing baryonic mass" discrepancy in galaxy clusters?
A: MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called "fuzzy dark matter."
B: MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20.
C: MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions.
D: MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 2.
E: MOND is a theory that eliminates the observed missing bar

  2%|▏         | 4/200 [00:00<00:35,  5.53it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Which of the following statements accurately describes the origin and significance of the triskeles symbol?
A: The triskeles symbol was reconstructed as a feminine divine triad by the rulers of Syracuse, and later adopted as an emblem. Its usage may also be related to the Greek name of Sicily, Trinacria, which means "having three headlands." The head of Medusa at the center of the Sicilian triskeles represents the three headlands.
B: The triskeles symbol is a representation of three interlinked spirals, which was adopted as an emblem by the rulers of Syracuse. Its usage in modern flags of Sicily has its origins in the ancient Greek name for the island, Trinacria, which means "Sicily with three corners." The head of Medusa at the center is a representation of the island's rich cultural heritage.
C: The triskeles symbol is a representation of a triple goddess, reconstructed by the rulers of Syracuse, who adopted it as an emblem. Its significance lies in the fact that it represents the Gr

  2%|▎         | 5/200 [00:00<00:34,  5.63it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Which of the following statements accurately depicts the relationship between Gauss's law, electric flux, electric field, and symmetry in electric fields?
A: Gauss's law holds only for situations involving symmetric electric fields, like those with spherical or cylindrical symmetry, and doesn't apply to other field types. Electric flux, as an expression of the total electric field passing through a closed surface, is influenced only by charges within the surface and unaffected by distant charges located outside it. The scalar quantity electric flux is strictly measured in SI fundamental quantities as kg·m3·s−3·A.
B: Gauss's law holds in all cases, but it is most useful for calculations involving symmetric electric fields, like those with spherical or cylindrical symmetry, as they allow for simpler algebraic manipulations. Electric flux is not affected by distant charges outside the closed surface, whereas the net electric field, E, can be influenced by any charges positioned outside of

  4%|▍         | 8/200 [00:01<00:37,  5.15it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  5%|▌         | 10/200 [00:01<00:26,  7.18it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Which of the following statements accurately describes the blocking temperature of an antiferromagnetic layer in a spin valve?
A: The blocking temperature of an antiferromagnetic layer in a spin valve is the temperature at which the magnetization of the ferromagnetic layer becomes aligned with the magnetic field. The blocking temperature is typically higher than the Néel temperature.
B: The blocking temperature of an antiferromagnetic layer in a spin valve is the temperature below which the layer loses its ability to "pin" the magnetization direction of an adjacent ferromagnetic layer. The blocking temperature is typically higher than the Néel temperature.
C: The blocking temperature of an antiferromagnetic layer in a spin valve is the temperature at which the ferromagnetic layer becomes completely demagnetized. The blocking temperature is typically higher than the Néel temperature.
D: The blocking temperature of an antiferromagnetic layer in a spin valve is the temperature at or above

  8%|▊         | 15/200 [00:02<00:27,  6.64it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
 12%|█▏        | 24/200 [00:03<00:20,  8.59it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
 16%|█▌        | 31/200 [00:04<00:27,  6.23it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
 18%|█▊        | 35/200 [00:05<00:24,  6.73it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
 18%|█▊        | 37/200 [00:05<00:25,  6.37it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
 21%|██        | 42/200 [00:06<00:20,  7.61it/s]Se

correct: 83 / 200


# Chain-of-Thoughts(CoT) Prompting

In [ ]:
import re
def process_text_output(output):
    # 1. 먼저 \boxed{} 형식의 답을 찾기
    result_output = re.findall(r'\\[bB]oxed\{\(?([A-E])\)?\}', output)

    # 2. 만약 \boxed{} 형식의 답이 없다면 "answer is" 패턴으로 찾기
    if not len(result_output):
        result_output = re.findall(r'[aA]nswer is\:?\s*\(?([A-E])\)?', output)

    # 3. 만약 answer is 형식의 답이 없다면 "answer:" 패턴으로 찾기
    if not len(result_output):
        result_output = re.findall(r'[aA]nswer:\s*\(?([A-E])\)?', output)

    # 4. 그래도 없다면 'A'로 설정
    if not len(result_output):
        print('No answer found')
        result_output = 'A'
    else:
        # 마지막으로 나온 정답 알파벳 선택
        result_output = result_output[-1]

    return result_output

In [ ]:
def process_df(df_part, gpu_id, preds_queue, answers_queue, MODEL_PATH):
    # Initialize model and tokenizer within the thread
    model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)
    model = model.to(f'cuda:{gpu_id}')
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    print("gpu id:", model.device)

    preds = []
    answers = []
    outputs = []
    for i in tqdm(range(len(df_part))):
        row = df_part.iloc[i]
        question = row['prompt']
        answer = row['answer']
        a, b, c, d, e = row['A'], row['B'], row['C'], row['D'], row['E']
        instruction = 'Please reason step-by-step to solve the following multiple-choice problem, and put your final answer(one of A, B, C, D, E) within \\boxed{}.'
        prompt = f'{instruction}\n{question}\nA: {a}\nB: {b}\nC: {c}\nD: {d}\nE: {e}'
        messages = [{"role": "user", "content": prompt}]
        prompt = tokenizer.apply_chat_template(messages, tokenize=False)

        # Generate prediction
        with torch.no_grad():
            inputs = tokenizer(prompt, return_tensors='pt').to(f'cuda:{gpu_id}')
            outputs_model = model.generate(**inputs, max_new_tokens=2048, do_sample=True, temperature=0.9, use_cache=True)
            pred = tokenizer.decode(outputs_model[0], skip_special_tokens=True)

        if i < 10:
            print(f'Prediction ({gpu_id}, {i}): {pred}')
            print(f'Ground Truth ({gpu_id}, {i}): {answer}')
        preds.append(process_text_output(pred))
        answers.append(answer)
    # Put results into queues
    preds_queue.put(preds)
    answers_queue.put(answers)

In [ ]:
#Naive Data Paralellism wtih multi-threading
from threading import Thread
from queue import Queue

# Split df into two parts
df1 = df.iloc[:100].reset_index(drop=True)
df2 = df.iloc[100:200].reset_index(drop=True)

# Create queues to collect results
preds_queue = Queue()
answers_queue = Queue()

# Define threads
t1 = Thread(target=process_df, args=(df1, 0, preds_queue, answers_queue, MODEL_PATH))
t2 = Thread(target=process_df, args=(df2, 1, preds_queue, answers_queue, MODEL_PATH))

# Start threads
t1.start()
t2.start()

# Wait for threads to finish
t1.join()
t2.join()

# Collect results
preds1 = preds_queue.get()
answers1 = answers_queue.get()
preds2 = preds_queue.get()
answers2 = answers_queue.get()

# Combine results
preds = preds1 + preds2
answers = answers1 + answers2

# Now compute correctness
print('correct:', (np.array(answers) == np.array(preds)).sum(), '/', len(df))

gpu id: cuda:1


  1%|          | 1/100 [00:06<11:01,  6.69s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prediction (1, 0): user

Please reason step-by-step to solve the following multiple-choice problem, and put your final answer(one of A, B, C, D, E) within \boxed{}.
What is a virtual particle?
A: A particle that is not affected by the strong force.
B: A particle that is not affected by the weak force.
C: A particle that is created in a laboratory for experimental purposes.
D: A particle that is not directly observable but is inferred from its effects on measurable particles.
E: A particle that is directly observable and can be measured in experiments.assistant

Let's break down the options step by step to solve this problem:

A) A particle that is not affected by the strong force. - The strong force is a fundamental force of nature that holds quarks together inside protons and neutrons and holds these particles together inside atomic nuclei. Particles that are not affected by the strong force are indeed virtual particles. 

B) A particle that is not affected by the weak force. - The we

  2%|▏         | 2/100 [00:07<04:53,  2.99s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prediction (1, 1): user

Please reason step-by-step to solve the following multiple-choice problem, and put your final answer(one of A, B, C, D, E) within \boxed{}.
Who proposed the principle of "complexity from noise" and when was it first introduced?
A: Ilya Prigogine in 1979
B: Henri Atlan in 1972
C: Democritus and Lucretius in ancient times
D: None of the above.
E: René Descartes in 1637assistant

The best answer is  (D):None of the above.
Ground Truth (1, 1): B


...


 84%|████████▍ | 84/100 [12:58<02:39,  9.97s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


KeyboardInterrupt: 

 85%|████████▌ | 85/100 [13:03<02:08,  8.58s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.

 86%|████████▌ | 86/100 [13:07<01:38,  7.06s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


No answer found


 88%|████████▊ | 88/100 [13:13<01:01,  5.15s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.

 59%|█████▉    | 59/100 [12:56<08:59, 13.16s/it]Exception in thread Thread-5 (process_df):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/tqdm/std.py", line 1191, in __iter__
    self.update(n - last_print_n)
  File "/opt/conda/lib/python3.10/site-packages/tqdm/std.py", line 1242, in update
    self.refresh(lock_args=self.lock_args)
  File "/opt/conda/lib/python3.10/site-packages/tqdm/std.py", line 1347, in refresh
    self.display()
  File "/opt/conda/lib/python3.10/site-packages/tqdm/std.py", line 1494, in display
    self.moveto(pos)
  File "/opt/conda/lib/python3.10/site-packages/tqdm/std.py", line 1443, in moveto
    self.fp.write('\n' * n + _term_move_up() * -n)
  File "/opt/conda/lib/python3.10/site-packages/tqdm/utils.py", line 196, in inner
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/ipy

In [ ]:
#TODO: add Self-Consistency!

# Retrieval Augmented Generation(RAG)

In [ ]:
class SentenceTransformer:
    def __init__(self, checkpoint, device="cuda:0"):
        self.device = device
        self.checkpoint = checkpoint
        self.model = AutoModel.from_pretrained(checkpoint).to(self.device).half()
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    def transform(self, batch):
        tokens = self.tokenizer(batch["text"], truncation=True, padding=True, return_tensors="pt", max_length=512)
        return tokens.to(self.device)

    def get_dataloader(self, sentences, batch_size=32):
        sentences = ["Represent this sentence for searching relevant passages: " + x for x in sentences]
        dataset = Dataset.from_dict({"text": sentences})
        dataset.set_transform(self.transform)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        return dataloader

    def encode(self, sentences, batch_size=32):
        dataloader = self.get_dataloader(sentences, batch_size=batch_size)
        embeddings = []
        for batch in tqdm(dataloader):
            with torch.no_grad():
                e = self.model(**batch).pooler_output
                e = F.normalize(e, p=2, dim=1)
                embeddings.append(e.detach().cpu().numpy())
        embeddings = np.concatenate(embeddings, axis=0)
        return embeddings

In [ ]:
import faiss
from torch.utils.data import DataLoader
from datasets import load_dataset, load_from_disk, Dataset
from pathlib import Path
import gc

dataset = load_from_disk("/kaggle/input/all-paraphs-parsed-expanded")
dataset

Dataset({
    features: ['title', 'section', 'text'],
    num_rows: 2101279
})

In [ ]:
print(dataset[0])
print(dataset[1])
print(dataset[2])

{'title': 'Russell Epstein', 'section': 'Russell Epstein', 'text': 'Russell Epstein is a professor of psychology at the University of Pennsylvania, who studies neural mechanisms underlying visual scene perception, event perception, object recognition, and spatial navigation in humans. His lab studies the role of the Parahippocampal and retrosplenial cortices in determining how people orient themselves relative to their surroundings.'}
{'title': 'Russell Epstein', 'section': 'Education', 'text': 'Epstein received an undergraduate degree in physics at the University of Chicago and a Ph.D. in computer vision with Alan Yuille at Harvard.'}
{'title': 'Ambient device', 'section': 'Ambient device', 'text': 'Ambient devices are a type of consumer electronics, characterized by their ability to be perceived at-a-glance, also known as "glanceable". Ambient devices use pre-attentive processing to display information and are aimed at minimizing mental effort. Associated fields include ubiquitous co

In [ ]:
# Load embedding model
RETRIEVER_PATH = '/kaggle/input/bge-small-faiss'
model = SentenceTransformer(RETRIEVER_PATH, device="cuda:0")

# Get embeddings of prompts
f = lambda row : " ".join([row["prompt"], str(row["A"]), str(row["B"]), str(row["C"]), str(row["D"]), str(row["E"])])
inputs = df.apply(f, axis=1).values # better results than prompt only
prompt_embeddings = model.encode(inputs)

# Search closest sentences in the wikipedia index
faiss_index = faiss.read_index(RETRIEVER_PATH + '/faiss.index')

search_index = faiss_index.search(np.float32(prompt_embeddings), k=5)[1]

# # FAISS 인덱스 생성
# d = prompt_embeddings.shape[1]  # 임베딩 벡터의 차원
# faiss_index = faiss.IndexFlatL2(d)  # L2 거리 기준의 인덱스 생성
# faiss_index.add(model.encode(your_numpy_array_of_texts))

dataset = load_from_disk("/kaggle/input/all-paraphs-parsed-expanded")
for i in range(len(df)):
    df.loc[i, "context"] = "-" + "\n-".join([dataset[int(j)]["text"] for j in search_index[i]])

# Free memory
faiss_index.reset()
del faiss_index, prompt_embeddings, model, dataset
torch.cuda.empty_cache()
gc.collect()

100%|██████████| 7/7 [00:00<00:00,  7.34it/s]


77

In [ ]:
df

,id,prompt,A,B,C,D,E,answer,context
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D,-MOND is an example of a class of theories kno...
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A,-In such systems we can define a certain time-...
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A,"-Classical Antiquity The triskeles proper, com..."
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C,-Regularization: Classical physics theory brea...
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D,-Several qualitative observations can be made ...
...,...,...,...,...,...,...,...,...,...
195,195,What is the relation between the three moment ...,The three moment theorem expresses the relatio...,The three moment theorem is used to calculate ...,The three moment theorem describes the relatio...,The three moment theorem is used to calculate ...,The three moment theorem is used to derive the...,C,-In civil engineering and structural analysis ...
196,196,"What is the throttling process, and why is it ...",The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,The throttling process is a steady adiabatic f...,The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,B,-Throttling One of the simple applications of ...
197,197,What happens to excess base metal as a solutio...,"The excess base metal will often solidify, bec...",The excess base metal will often crystallize-o...,"The excess base metal will often dissolve, bec...","The excess base metal will often liquefy, beco...","The excess base metal will often evaporate, be...",B,"-Similarly, a hypoeutectoid alloy has two crit..."
198,198,"What is the relationship between mass, force, ...",Mass is a property that determines the weight ...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is a property that determines the size of...,D,-Mass is (among other properties) an inertial ...


In [ ]:
# vllm example for efficient generation
# import vllm

# llm = vllm.LLM(
#     "/kaggle/input/llama-3.2/transformers/1b/1", #Not supported yet
#     quantization="awq",
#     tensor_parallel_size=2,
#     gpu_memory_utilization=0.95,
#     trust_remote_code=True,
#     dtype="half",
#     enforce_eager=True,
#     max_model_len=1024,
#     #distributed_executor_backend="ray",
# )
# tokenizer = llm.get_tokenizer()
# responses = llm.generate(
#     all_prompts,
#     vllm.SamplingParams(
#         n=1,  # Number of output sequences to return for each prompt.
#         top_p=0.9,  # Float that controls the cumulative probability of the top tokens to consider.
#         temperature=0,  # randomness of the sampling
#         seed=42, # Seed for reprodicibility
#         skip_special_tokens=True,  # Whether to skip special tokens in the output.
#         max_tokens=1024,  # Maximum number of tokens to generate per output sequence.
#         logprobs=5
#     ),
#     use_tqdm = True
# )

In [ ]:
def process_df(df_part, gpu_id, preds_queue, answers_queue, MODEL_PATH):
    # Initialize model and tokenizer within the thread
    model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)
    model = model.to(f'cuda:{gpu_id}')
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    print("gpu id:", model.device)

    preds = []
    answers = []
    outputs = []
    for i in tqdm(range(len(df_part))):
        row = df_part.iloc[i]
        question = row['prompt']
        #context추가!
        context = row['context']
        answer = row['answer']
        a, b, c, d, e = row['A'], row['B'], row['C'], row['D'], row['E']
        instruction = 'Please reason step-by-step to solve the following multiple-choice problem, and put your final answer(one of A, B, C, D, E) within \\boxed{}. And please refer to the given contexts which are possibly related to the question.'
        prompt = f'{instruction}\n\nQuestion: {question}\n\nContext(s): {context}\n\n(A): {a}\n(B): {b}\n(C): {c}\n(D): {d}\n(E): {e}\n'
        messages = [{"role": "user", "content": prompt}]
        prompt = tokenizer.apply_chat_template(messages, tokenize=False)

        # Generate prediction
        with torch.no_grad():
            inputs = tokenizer(prompt, return_tensors='pt').to(f'cuda:{gpu_id}')
            outputs_model = model.generate(**inputs, max_new_tokens=2048, do_sample=True, temperature=0.9)
            pred = tokenizer.decode(outputs_model[0], skip_special_tokens=True)

        if i < 10:
            print(f'Prediction ({gpu_id}, {i}): {pred}')
            print(f'Ground Truth ({gpu_id}, {i}): {answer}')
        preds.append(process_text_output(pred))
        answers.append(answer)
    # Put results into queues
    preds_queue.put(preds)
    answers_queue.put(answers)

In [ ]:
#Naive Data Paralellism wtih multi-threading
from threading import Thread
from queue import Queue
from joblib import Parallel, delayed


# Split df into two parts
df1 = df.iloc[:100].reset_index(drop=True)
df2 = df.iloc[100:200].reset_index(drop=True)

# Create queues to collect results
preds_queue = Queue()
answers_queue = Queue()

# Define threads
t1 = Thread(target=process_df, args=(df1, 0, preds_queue, answers_queue, MODEL_PATH))
t2 = Thread(target=process_df, args=(df2, 1, preds_queue, answers_queue, MODEL_PATH))

# Start threads
t1.start()
t2.start()

# Wait for threads to finish
t1.join()
t2.join()

# Collect results
preds1 = preds_queue.get()
answers1 = answers_queue.get()
preds2 = preds_queue.get()
answers2 = answers_queue.get()

# Combine results
preds = preds1 + preds2
answers = answers1 + answers2

# Now compute correctness
print('correct:', (np.array(answers) == np.array(preds)).sum(), '/', len(df))

In [ ]:
additional_df = pd.read_csv('/kaggle/input/15k-high-quality-examples/5900_examples.csv')
# Convert all columns to string type
additional_df = additional_df.astype(str)
additional_df

,prompt,B,D,C,E,A,answer
0,What is the primary focus of the television pr...,Exploring the history and evolution of British...,Showcasing a wide range of television programm...,Investigating the effects of excessive televis...,Analyzing the impact of excessive television c...,Examining the different genres and formats of ...,D
1,When was the Navy Midshipmen baseball team fir...,1905,1945,1957,1920,1893,A
2,"How many volumes of the manga series ""Prince F...",Ten volumes,Three volumes,Seven volumes,One volume,Five volumes,B
3,"In the study of diffraction, what is the relat...",The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,C
4,Which two counties does the A4113 road pass th...,Powys and Herefordshire,Knighton and Bromfield,Powys and Shropshire,Herefordshire and Bromfield,Shropshire and Knighton,B
...,...,...,...,...,...,...,...
5895,"What is the content of the album ""Homunculus P...",The album comprises four of Frohmader's albums...,The album is a collection of live performances...,The album consists of Frohmader's biggest hits...,The album includes three of Frohmader's albums...,The album is a compilation of Frohmader's earl...,E
5896,What is the population of Cameron Township in ...,135,150,100,175,200,B
5897,Which region is the native habitat of Aesculus...,The Great Plains of the United States.,The Ohio Valley and Appalachian Mountains of t...,The Rocky Mountains of the United States.,The Pacific Northwest of the United States.,The Gulf Coast region of the United States.,D
5898,What award did Mariangela Melato win for her r...,Mariangela Melato was awarded the Academy Awar...,Mariangela Melato was awarded the Palme d'Or f...,Mariangela Melato was awarded the David di Don...,Mariangela Melato was awarded the Golden Globe...,Mariangela Melato was awarded the BAFTA for Be...,C


# RAG + Parameter Efficient Fine Tuning(PEFT)

In [ ]:
# Load embedding model
RETRIEVER_PATH = '/kaggle/input/bge-small-faiss'
model = SentenceTransformer(RETRIEVER_PATH, device="cuda:0")

# Get embeddings of prompts
f = lambda row : " ".join([row["prompt"], row["A"], row["B"], row["C"], row["D"], row["E"]])
inputs = additional_df.apply(f, axis=1).values # better results than prompt only
prompt_embeddings = model.encode(inputs)

# Search closest sentences in the wikipedia index
faiss_index = faiss.read_index(RETRIEVER_PATH + '/faiss.index')

search_index = faiss_index.search(np.float32(prompt_embeddings), k=5)[1]

# # FAISS 인덱스 생성
# d = prompt_embeddings.shape[1]  # 임베딩 벡터의 차원
# faiss_index = faiss.IndexFlatL2(d)  # L2 거리 기준의 인덱스 생성
# faiss_index.add(model.encode(your_numpy_array_of_texts))

dataset = load_from_disk("/kaggle/input/all-paraphs-parsed-expanded")
for i in range(len(additional_df)):
    additional_df.loc[i, "context"] = "-" + "\n-".join([dataset[int(j)]["text"] for j in search_index[i]])

# Free memory
faiss_index.reset()
del faiss_index, prompt_embeddings, model, dataset
torch.cuda.empty_cache()
gc.collect()

100%|██████████| 185/185 [00:08<00:00, 21.57it/s]


0

In [ ]:
# quantization_config = BitsAndBytesConfig(load_in_8bit=True) #8-bit quantization
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit = True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_use_double_quant=True,
# ) #4-bit quantization

MODEL_PATH = '/kaggle/input/llama-3.2/transformers/1b/1'

# model = AutoModelForMultipleChoice.from_pretrained(...)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, torch_dtype=torch.float16) #attn_implementation="flash_attention_2", quantization_config=quantization_config
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [ ]:
print(tokenizer.all_special_tokens)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.get_vocab()['<|end_of_text|>']
tokenizer.pad(tokenizer(['hihihi','hoo']))

['<|begin_of_text|>', '<|end_of_text|>']


{'input_ids': [[6151, 6151, 6151], [119820, 128001, 128001]], 'attention_mask': [[1, 1, 1], [1, 0, 0]]}

In [ ]:
def preprocess(example, max_context_len=1024, tokenizer=tokenizer):
    first_sentence = f"{example['context']}"
    second_sentences = f"\n ### {example['prompt']} \nA: {example['A']}\nB: {example['D']}\nC: {example['C']}\nD: {example['D']}\nE: {example['E']}\n\n ### Answer: {example['answer']}" #### " + example['prompt'] + f" {eos_token} " + example[option] + f" {eos_token}" for option in 'ABCDE']
    try:
        tokenized_example = tokenizer(first_sentence, second_sentences, truncation='only_first',
                                      max_length=max_context_len, add_special_tokens=False)
    except:
        tokenized_example = tokenizer(second_sentences, truncation=True, max_length=max_context_len, add_special_tokens=False)
    return tokenized_example

In [ ]:
dataset = Dataset.from_pandas(additional_df)
tokenized_dataset = dataset.map(preprocess, remove_columns=['prompt', 'context', 'A', 'B', 'C', 'D', 'E', 'answer'])

Map:   0%|          | 0/5900 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
    r=8, #rank of low-rank matrices, low rank -> low VRAM usage
    lora_alpha=4,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj"], #modules to apply LORA
    # modules_to_save=['lm_head'], #modules to finetune without LORA
    inference_mode=False,
)
model = get_peft_model(model, peft_config)
print(model.model.model.embed_tokens.weight.requires_grad)
print(model.model.lm_head.weight.requires_grad)
# model.model.embed_tokens.requires_grad = False
model.print_trainable_parameters()

False
False
trainable params: 851,968 || all params: 1,236,666,368 || trainable%: 0.0689


In [ ]:
from trl import SFTTrainer,SFTConfig,DataCollatorForCompletionOnlyLM

training_args = SFTConfig(
    warmup_ratio=0.1,
    learning_rate=1e-4,
    per_device_train_batch_size=2,
    # per_device_eval_batch_size=2,
    num_train_epochs=1,
    report_to='none',
    output_dir = f'./checkpoints',
    overwrite_output_dir=True,
    fp16=True,
    # bf16=True,
    optim='paged_adamw_8bit', #'paged_adamw_32bit', 'paged_adamw_8bit', 'adamw_torch'
    # torch_compile=True, #model = torch.compile(model)
    gradient_accumulation_steps=4,
    logging_steps=25,
    # evaluation_strategy='steps',
    # eval_steps=10,
    save_strategy="steps",
    save_steps=10,
    lr_scheduler_type='cosine',
    weight_decay=0.01,
    save_total_limit=2,
)

In [ ]:
torch.cuda.empty_cache()
gc.collect()
libc.malloc_trim(0)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    tokenizer=tokenizer,
    peft_config=peft_config,
    max_seq_length=1024,
    data_collator=DataCollatorForCompletionOnlyLM(tokenizer.encode(" ### Answer:", add_special_tokens = False), tokenizer=tokenizer)
)

trainer.train()
trainer.save_model(f'model')

Step,Training Loss
25,1.873700
50,1.742900
75,1.619700
100,1.611500
125,1.601100
150,1.604300
175,1.603300
200,1.591500
225,1.606100
250,1.606100


In [ ]:
#Next Token Predicton + greedy decoding
preds = []
answers = []
for i in tqdm(range(len(df))):
    example = df.iloc[i]
    # print(row.values)
    answer = example['answer']
    # prompt = f"{row['context']}\n ### {row['prompt']} \nA: {row['A']}\nB: {row['D']}\nC: {row['C']}\nD: {row['D']}\nE: {row['E']}\n\n### Answer:"
    first_sentence = f"{example['context']}"
    second_sentences = f"\n ### {example['prompt']} \nA: {example['A']}\nB: {example['D']}\nC: {example['C']}\nD: {example['D']}\nE: {example['E']}\n\n ### Answer:" #### " + example['prompt'] + f" {eos_token} " + example[option] + f" {eos_token}" for option in 'ABCDE']
    try:
        tokenized_example = tokenizer(first_sentence, second_sentences, truncation='only_first',
                                      max_length=1024, add_special_tokens=False, return_tensors='pt')
    except:
        tokenized_example = tokenizer(second_sentences, truncation=True, max_length=1024, add_special_tokens=False, return_tensors='pt')

    inputs = tokenized_example.to(model.device)
    pred = model.generate(**inputs, max_new_tokens=1, do_sample=False)[0]
    pred = tokenizer.decode(pred)
    if i < 10:
        print(pred)
        print(f'Ground Truth: {answer}')
    preds.append(pred[-1])
    answers.append(answer)
print('correct:', (np.array(answers) == np.array(preds)).sum(), '/', len(df))

  1%|          | 2/200 [00:00<00:31,  6.35it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


-MOND is an example of a class of theories known as modified gravity, and is an alternative to the hypothesis that the dynamics of galaxies are determined by massive, invisible dark matter halos. Since Milgrom's original proposal, proponents of MOND have claimed to successfully predict a variety of galactic phenomena that they state are difficult to understand as consequences of dark matter.Though MOND explains the anomalously great rotational velocities of galaxies at their perimeters, it does not fully explain the velocity dispersions of individual galaxies within galaxy clusters. MOND reduces the discrepancy between the velocity dispersions and clusters' observed missing baryonic mass from a factor of around 10 to a factor of about 2. However, the residual discrepancy cannot be accounted for by MOND, requiring that other explanations close the gap such as the presence of as-yet undetected missing baryonic matter.The accurate measurement of the speed of gravitational waves compared t

  2%|▏         | 4/200 [00:00<00:29,  6.70it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


-Classical Antiquity The triskeles proper, composed of three human legs, is younger than the triple spiral, found in decorations on Greek pottery especially as a design shown on hoplite shields, and later also minted on Greek and Anatolian coinage.
An early example is found on the shield of Achilles in an Attic hydria of the late 6th century BCE.
It is found on coinage in Lycia, and on staters of Pamphylia (at Aspendos, 370–333 BCE) and Pisidia. The meaning of the Greek triskeles is not recorded directly.
The Duc de Luynes in his 1835 study noted the co-occurrence of the symbol with the eagle, the cockerel, the head of Medusa, Perseus, three crescent moons, three ears of corn, and three grains of corn.
From this, he reconstructed feminine divine triad which he identified with the "triple goddess" Hecate.
The triskeles was adopted as emblem by the rulers of Syracuse. It is possible that this usage is related with the Greek name of the island of Sicily, Trinacria (Τρινακρία 'having three

  3%|▎         | 6/200 [00:00<00:28,  6.81it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

-An inductive dimension may be defined inductively as follows. Consider a discrete set of points (such as a finite collection of points) to be 0-dimensional. By dragging a 0-dimensional object in some direction, one obtains a 1-dimensional object. By dragging a 1-dimensional object in a new direction, one obtains a 2-dimensional object. In general one obtains an (n + 1)-dimensional object by dragging an n-dimensional object in a new direction. The inductive dimension of a topological space may refer to the small inductive dimension or the large inductive dimension, and is based on the analogy that, in the case of metric spaces, (n + 1)-dimensional balls have n-dimensional boundaries, permitting an inductive definition based on the dimension of the boundaries of open sets. Moreover, the boundary of a discrete set of points is the empty set, and therefore the empty set can be taken to have dimension -1.Similarly, for the class of CW complexes, the dimension of an object is the largest n 

  5%|▌         | 10/200 [00:01<00:25,  7.60it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


-The interactions and phenomena summarized in the subjects of radiative transfer and physical optics can result in shifts in the wavelength and frequency of electromagnetic radiation. In such cases, the shifts correspond to a physical energy transfer to matter or other photons rather than being by a transformation between reference frames. Such shifts can be from such physical phenomena as coherence effects or the scattering of electromagnetic radiation whether from charged elementary particles, from particulates, or from fluctuations of the index of refraction in a dielectric medium as occurs in the radio phenomenon of radio whistlers. While such phenomena are sometimes referred to as "redshifts" and "blueshifts", in astrophysics light-matter interactions that result in energy shifts in the radiation field are generally referred to as "reddening" rather than "redshifting" which, as a term, is normally reserved for the effects discussed above.In many circumstances scattering causes rad

 10%|█         | 21/200 [00:02<00:24,  7.42it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
 38%|███▊      | 76/200 [00:10<00:17,  7.09it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
 42%|████▏     | 84/200 [00:11<00:15,  7.38it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
 44%|████▎     | 87/200 [00:12<00:14,  7.96it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
 45%|████▌     | 90/200 [00:12<00:13,  8.07it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
 57%|█████▊    | 115/200 [00:16<00:12,  7.05it/s]S

correct: 68 / 200


Pretrained LLM = 이미 요리 잘하는 셰프

Fine-tuning = 이 셰프에게 “우리 식당 메뉴 스타일”을 다시 훈련

LoRAConfig = “셰프에게 새로운 기술을 어디에 추가할지” (예: 손동작만 훈련)

SFTConfig = “훈련 과정은 어떻게 할지” (예: 하루 3시간 연습, 1주일, 점검은 매 10분마다)